In [18]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [23]:
import os
import pandas as pd
import glob
import pickle

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_all_systems, preprocess_all_models

In [76]:
models = [os.path.basename(p) for p in glob.glob(f'../data/models/{set_}/*')]

dfs = []

for model in models:
    if os.path.isfile(f'../data/models/{set_}/{model}/system_evaluation.csv'):
        df_ = pd.read_csv(f'../data/models/{set_}/{model}/system_evaluation.csv', index_col=['subset', 'references', 'metric'])
        dfs.append(df_)

scores_df = pd.concat(dfs, keys=models).reset_index(level=[1, 2], drop=True).unstack()
scores_df.columns = scores_df.columns.droplevel()

scores_df

metric,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
-2544668248973508035,59.23,88.3,68.9,53.4,41.4,0.442593,0.367464
-3107892160950639319,57.88,87.8,67.9,51.9,39.7,0.438597,0.382481
-4714147435136210209,59.05,89.1,69.9,54.4,42.3,0.438176,0.366733
-4954308565556340093,59.47,88.6,69.1,53.6,41.7,0.443446,0.365709
-5814854809871667593,58.29,87.9,68.2,52.5,40.4,0.438669,0.382676
-7318950268519836208,57.80,88.3,67.8,51.9,39.9,0.438994,0.381506
-7939323232597032699,58.69,88.6,69.5,53.9,41.9,0.435792,0.377020
-8297383916663440440,59.02,89.6,70.1,54.7,42.4,0.438532,0.361711
2898124432584246300,59.51,88.6,69.3,53.8,41.8,0.442874,0.364783
3361903896355688010,59.49,88.8,69.3,53.8,41.7,0.442555,0.365563


In [77]:
params_dfs = []
for model in models:
    with open(f'../data/models/dev/{model}/params.pkl', 'rb') as f:
        params_dfs.append(pd.DataFrame([pickle.load(f)], index=[model]))
    

params_df = pd.concat(params_dfs)

params_df

,dp_scorer,fallback_template,max_dp,max_sa,max_tems,referrer,sa_scorer,tems_lm_bos,tems_lm_eos,tems_lm_n,tems_lm_name,tems_lm_preprocess_input,txs_lm_bos,txs_lm_eos,txs_lm_n,txs_lm_name,txs_lm_preprocess_input
-2544668248973508035,ltr_lasso,jjt,3,4,5,counter,ltr_lasso,False,False,6,lower,lower,False,False,6,lower,lower
-3107892160950639319,random,jjt,3,4,5,counter,random,False,False,6,lower,lower,False,False,6,lower,lower
-4714147435136210209,ltr_lasso,jjt,3,4,5,counter,ltr_lasso,False,False,3,lower,lower,False,False,3,lower,lower
-4954308565556340093,ltr_lasso,jjt,3,4,5,counter,random,False,False,6,lower,lower,False,False,6,lower,lower
-5814854809871667593,random,jjt,3,4,5,counter,ltr_lasso,False,False,6,lower,lower,False,False,6,lower,lower
-7318950268519836208,random,jjt,3,4,5,counter,random,False,False,3,lower,lower,False,False,6,lower,lower
-7939323232597032699,random,jjt,3,4,5,counter,ltr_lasso,False,False,3,lower,lower,False,False,3,lower,lower
-8297383916663440440,ltr_lasso,jjt,3,4,5,counter,random,False,False,6,lower,lower,False,False,3,lower,lower
2898124432584246300,ltr_lasso,jjt,3,4,5,counter,ltr_lasso,False,False,3,lower,lower,False,False,6,lower,lower
3361903896355688010,ltr_lasso,jjt,3,4,5,counter,random,False,False,3,lower,lower,False,False,6,lower,lower


In [78]:
df = pd.merge(scores_df, params_df, left_index=True, right_index=True)

cols = ['bleu', 'meteor', 'ter', 'dp_scorer', 'sa_scorer', 'tems_lm_n', 'txs_lm_n']
df.sort_values('bleu', ascending=False).loc[:, cols]

,bleu,meteor,ter,dp_scorer,sa_scorer,tems_lm_n,txs_lm_n
2898124432584246300,59.51,0.442874,0.364783,ltr_lasso,ltr_lasso,3,6
3361903896355688010,59.49,0.442555,0.365563,ltr_lasso,random,3,6
-4954308565556340093,59.47,0.443446,0.365709,ltr_lasso,random,6,6
5167415708380055249,59.34,0.439778,0.359859,ltr_lasso,random,3,3
-2544668248973508035,59.23,0.442593,0.367464,ltr_lasso,ltr_lasso,6,6
6598774345999131478,59.20,0.440458,0.378824,random,ltr_lasso,3,6
-4714147435136210209,59.05,0.438176,0.366733,ltr_lasso,ltr_lasso,3,3
-8297383916663440440,59.02,0.438532,0.361711,ltr_lasso,random,6,3
6267418638451139054,58.91,0.437511,0.365173,ltr_lasso,ltr_lasso,6,3
-7939323232597032699,58.69,0.435792,0.377020,random,ltr_lasso,3,3
